In [1]:
from utilities import get_all_species, get_compositional_features
import os

import torch
import ase.io
import numpy as np
from multiprocessing import cpu_count
from pathos.multiprocessing import ProcessingPool as Pool
from tqdm import tqdm
import torch_geometric
from torch_geometric.data import Data
from torch_geometric.loader import DataLoader, DataListLoader
from torch import nn
import torch.nn.functional as F
from sklearn.linear_model import Ridge
from utilities import ModelKeeper
import time
from scipy.spatial.transform import Rotation
from torch.optim.lr_scheduler import LambdaLR
import sys
import copy
import inspect
import yaml
from torch_geometric.nn import DataParallel

from molecule import Molecule, batch_to_dict
from hypers import Hypers
from pet import PET
from utilities import FullLogger
from utilities import get_rmse, get_mae, get_relative_rmse, get_loss
from analysis import get_structural_batch_size, convert_atomic_throughput

In [2]:
EPSILON = 1e-10
STRUCTURES_PATH = '../datasets/coll/coll_v1.2_AE_test.xyz'
DEFAULT_HYPERS_PATH = 'default_hypers.yaml'
SP_HYPERS_PATH = 'sp_default_hypers.yaml'
NUM_MOLECULES = 4

In [3]:
hypers = Hypers()
hypers.load_from_file(DEFAULT_HYPERS_PATH)

hypers_only_length = copy.deepcopy(hypers)
hypers_only_length.USE_ONLY_LENGTH = True

In [4]:
structures = ase.io.read(STRUCTURES_PATH, index = f':{NUM_MOLECULES}')
all_species = get_all_species(structures)
molecules = [Molecule(structure, hypers.R_CUT, hypers.USE_ADDITIONAL_SCALAR_ATTRIBUTES, hypers.USE_FORCES) for structure in tqdm(structures)]

max_nums = [molecule.get_max_num() for molecule in molecules]
max_num = np.max(max_nums)
graphs = [molecule.get_graph(max_num, all_species) for molecule in tqdm(molecules)]
loader = DataLoader(graphs, batch_size=10, shuffle=False)

100%|███████████████████████████████████████████| 4/4 [00:00<00:00, 1277.58it/s]


In [5]:
def get_model(hypers):
    add_tokens = []
    for _ in range(hypers.N_GNN_LAYERS - 1):
        add_tokens.append(hypers.ADD_TOKEN_FIRST)
    add_tokens.append(hypers.ADD_TOKEN_SECOND)

    model = PET(hypers, hypers.TRANSFORMER_D_MODEL, hypers.TRANSFORMER_N_HEAD,
                           hypers.TRANSFORMER_DIM_FEEDFORWARD, hypers.N_TRANS_LAYERS, 
                           0.0, len(all_species), 
                           hypers.N_GNN_LAYERS, hypers.HEAD_N_NEURONS, hypers.TRANSFORMERS_CENTRAL_SPECIFIC, hypers.HEADS_CENTRAL_SPECIFIC, 
                           add_tokens).cuda()

    model.augmentation = False
    return model

model = get_model(hypers)
model_only_length = get_model(hypers_only_length)

In [6]:
def compute_for_random_rotation(model, is_sp = False):
    structures = ase.io.read(STRUCTURES_PATH, index = f':{NUM_MOLECULES}')
    
    rotation = Rotation.random(1).as_matrix()[0]
    for struc in structures:
        struc.positions = np.dot(struc.positions, rotation)
    
    all_species = get_all_species(structures)
    molecules = [Molecule(structure, hypers.R_CUT, hypers.USE_ADDITIONAL_SCALAR_ATTRIBUTES, hypers.USE_FORCES) for structure in structures]

    max_nums = [molecule.get_max_num() for molecule in molecules]
    max_num = np.max(max_nums)
    graphs = [molecule.get_graph(max_num, all_species) for molecule in molecules]
    loader = DataLoader(graphs, batch_size=1, shuffle=False)
    
    torch.set_printoptions(precision=10)
    result = []
    for batch in loader:
        batch.cuda()
        if is_sp:
            _, _, _, predictions_energies, _, _, _ = model(batch)
        else:
            predictions_energies, _, _, _ = model(batch)
        result.append(predictions_energies)
    result = torch.cat(result, dim = 0)
    print(result)
    
    

# Normal model is not invariant with respect to rotations:

In [7]:
        
for _ in range(10):
    compute_for_random_rotation(model)

tensor([ 4.6532812119,  1.1738374233, 11.0723333359,  3.0509610176],
       device='cuda:0', grad_fn=<CatBackward0>)
tensor([ 3.8727846146,  1.1035382748, 11.1587944031,  2.5032422543],
       device='cuda:0', grad_fn=<CatBackward0>)
tensor([ 4.5295562744,  1.2187290192, 12.2790145874,  2.7994041443],
       device='cuda:0', grad_fn=<CatBackward0>)
tensor([ 3.9224903584,  1.1699337959, 11.3691978455,  2.5340712070],
       device='cuda:0', grad_fn=<CatBackward0>)
tensor([ 4.4406013489,  0.9452638626, 10.7616453171,  3.1007592678],
       device='cuda:0', grad_fn=<CatBackward0>)
tensor([ 4.3224363327,  1.1158511639, 10.8621082306,  3.0472834110],
       device='cuda:0', grad_fn=<CatBackward0>)
tensor([ 3.8266706467,  1.1204146147, 10.6523494720,  2.4459931850],
       device='cuda:0', grad_fn=<CatBackward0>)
tensor([ 4.0556879044,  1.2029281855, 11.9173603058,  2.6341493130],
       device='cuda:0', grad_fn=<CatBackward0>)
tensor([ 4.5633668900,  1.1726646423, 11.6072378159,  2.95490169

# Only length model is invariant with respect to rotations

In [8]:
for _ in range(10):
    compute_for_random_rotation(model_only_length)

tensor([-19.0117378235,  -7.5118646622, -47.4766616821, -12.1423044205],
       device='cuda:0', grad_fn=<CatBackward0>)
tensor([-19.0117378235,  -7.5118656158, -47.4766654968, -12.1423053741],
       device='cuda:0', grad_fn=<CatBackward0>)
tensor([-19.0117397308,  -7.5118656158, -47.4766654968, -12.1423053741],
       device='cuda:0', grad_fn=<CatBackward0>)
tensor([-19.0117378235,  -7.5118665695, -47.4766654968, -12.1423053741],
       device='cuda:0', grad_fn=<CatBackward0>)
tensor([-19.0117397308,  -7.5118656158, -47.4766654968, -12.1423053741],
       device='cuda:0', grad_fn=<CatBackward0>)
tensor([-19.0117397308,  -7.5118656158, -47.4766654968, -12.1423053741],
       device='cuda:0', grad_fn=<CatBackward0>)
tensor([-19.0117378235,  -7.5118646622, -47.4766616821, -12.1423053741],
       device='cuda:0', grad_fn=<CatBackward0>)
tensor([-19.0117378235,  -7.5118656158, -47.4766616821, -12.1423053741],
       device='cuda:0', grad_fn=<CatBackward0>)
tensor([-19.0117397308,  -7.5118

# Symmetrization protocol

In [9]:
from sp_frames_calculator import SPFramesCalculator
from pet_sp import PETSP
sp_hypers = Hypers()
sp_hypers.load_from_file(SP_HYPERS_PATH)
sp_frames_calculator = SPFramesCalculator(sp_hypers)

/home/sergey/final_reproduce/pet_latest/sp_frames_calculator.py:126: ComplexWarning: Casting complex values to real discards the imaginary part
  self.lambert_constant = torch.tensor(float(lambertw(np.exp(-1.0))))


# symmetrization of normal model

In [10]:
model_sp = PETSP(model, None,
                 100.0, hypers.USE_ENERGIES, hypers.USE_FORCES,
                 sp_frames_calculator, 50,
                 epsilon = EPSILON).cuda()

for _ in range(10):
    compute_for_random_rotation(model_sp, is_sp = True)

tensor([ 4.3400225639,  1.0897265673, 11.2385025024,  2.8997604847],
       device='cuda:0', grad_fn=<CatBackward0>)
tensor([ 4.3400220871,  1.0897264481, 11.2385025024,  2.8997597694],
       device='cuda:0', grad_fn=<CatBackward0>)
tensor([ 4.3400225639,  1.0897266865, 11.2385025024,  2.8997602463],
       device='cuda:0', grad_fn=<CatBackward0>)
tensor([ 4.3400220871,  1.0897265673, 11.2385044098,  2.8997609615],
       device='cuda:0', grad_fn=<CatBackward0>)
tensor([ 4.3400225639,  1.0897266865, 11.2385034561,  2.8997600079],
       device='cuda:0', grad_fn=<CatBackward0>)
tensor([ 4.3400230408,  1.0897265673, 11.2385025024,  2.8997604847],
       device='cuda:0', grad_fn=<CatBackward0>)
tensor([ 4.3400225639,  1.0897264481, 11.2385015488,  2.8997607231],
       device='cuda:0', grad_fn=<CatBackward0>)
tensor([ 4.3400220871,  1.0897264481, 11.2384996414,  2.8997602463],
       device='cuda:0', grad_fn=<CatBackward0>)
tensor([ 4.3400225639,  1.0897262096, 11.2385072708,  2.89976072

# model combined of normal one and only length as auxiliary

In [11]:
model_sp = PETSP(model, model_only_length,
                 100.0, hypers.USE_ENERGIES, hypers.USE_FORCES,
                 sp_frames_calculator, 50,
                 epsilon = EPSILON).cuda()

for _ in range(10):
    compute_for_random_rotation(model_sp, is_sp = True)

tensor([ 4.3400225639,  1.0897265673, 11.2385034561,  2.8997609615],
       device='cuda:0', grad_fn=<CatBackward0>)
tensor([ 4.3400225639,  1.0897264481, 11.2385025024,  2.8997607231],
       device='cuda:0', grad_fn=<CatBackward0>)
tensor([ 4.3400225639,  1.0897264481, 11.2385063171,  2.8997604847],
       device='cuda:0', grad_fn=<CatBackward0>)
tensor([ 4.3400220871,  1.0897266865, 11.2385044098,  2.8997604847],
       device='cuda:0', grad_fn=<CatBackward0>)
tensor([ 4.3400225639,  1.0897265673, 11.2385015488,  2.8997602463],
       device='cuda:0', grad_fn=<CatBackward0>)
tensor([ 4.3400230408,  1.0897265673, 11.2385015488,  2.8997604847],
       device='cuda:0', grad_fn=<CatBackward0>)
tensor([ 4.3400225639,  1.0897263288, 11.2385025024,  2.8997602463],
       device='cuda:0', grad_fn=<CatBackward0>)
tensor([ 4.3400225639,  1.0897264481, 11.2385005951,  2.8997609615],
       device='cuda:0', grad_fn=<CatBackward0>)
tensor([ 4.3400225639,  1.0897264481, 11.2385044098,  2.89976048

# symmetrization protocol with additional augmentations

In [12]:
additional_rotations = [torch.FloatTensor(el) for el in Rotation.random(5).as_matrix()]

model_sp = PETSP(model, None,
                 100.0, hypers.USE_ENERGIES, hypers.USE_FORCES,
                 sp_frames_calculator, 50,
                 additional_rotations = additional_rotations, 
                 epsilon = EPSILON).cuda()

for _ in range(10):
    compute_for_random_rotation(model_sp, is_sp = True)

tensor([ 4.3347034454,  1.1222932339, 11.3358707428,  2.8711168766],
       device='cuda:0', grad_fn=<CatBackward0>)
tensor([ 4.3347039223,  1.1222932339, 11.3358716965,  2.8711166382],
       device='cuda:0', grad_fn=<CatBackward0>)
tensor([ 4.3347039223,  1.1222934723, 11.3358726501,  2.8711175919],
       device='cuda:0', grad_fn=<CatBackward0>)
tensor([ 4.3347039223,  1.1222929955, 11.3358707428,  2.8711168766],
       device='cuda:0', grad_fn=<CatBackward0>)
tensor([ 4.3347034454,  1.1222934723, 11.3358707428,  2.8711166382],
       device='cuda:0', grad_fn=<CatBackward0>)
tensor([ 4.3347039223,  1.1222935915, 11.3358736038,  2.8711175919],
       device='cuda:0', grad_fn=<CatBackward0>)
tensor([ 4.3347029686,  1.1222933531, 11.3358726501,  2.8711168766],
       device='cuda:0', grad_fn=<CatBackward0>)
tensor([ 4.3347034454,  1.1222929955, 11.3358726501,  2.8711166382],
       device='cuda:0', grad_fn=<CatBackward0>)
tensor([ 4.3347029686,  1.1222933531, 11.3358726501,  2.87111711

In [13]:
print(additional_rotations)

[tensor([[ 0.3888399899,  0.8029066920, -0.4518232942],
        [ 0.7725714445, -0.5513450503, -0.3148841560],
        [-0.5019331574, -0.2266262174, -0.8346877694]]), tensor([[ 0.3473148942, -0.3901394606,  0.8527388573],
        [ 0.7879478335,  0.6144540310, -0.0398049764],
        [-0.5084393620,  0.6857386231,  0.5208185911]]), tensor([[-0.1875014454, -0.2049000263,  0.9606556296],
        [ 0.6737301350, -0.7385192513, -0.0260210168],
        [ 0.7147943377,  0.6423436403,  0.2765207291]]), tensor([[ 0.5231038332, -0.8319866657, -0.1848256886],
        [-0.4213648140, -0.4409741461,  0.7924603820],
        [-0.7408198118, -0.3366600573, -0.5812451839]]), tensor([[ 0.8584657311,  0.0436591953,  0.5110092163],
        [-0.0621182807,  0.9978860617,  0.0190985128],
        [-0.5090951324, -0.0481384322,  0.8593630195]])]
